In [1]:
import os
os.chdir(r"C:\Users\cdron\Desktop\Cleanup\Misc\kaggle\West_Nile_Virus_Pred")
os.getcwd()

'C:\\Users\\cdron\\Desktop\\Cleanup\\Misc\\kaggle\\West_Nile_Virus_Pred'

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import csv
import gzip

import numpy as np
import pandas as pd
from dateutil.parser import parse

In [3]:
WEATHER_VARS_WITH_M_T = (u'Tmax', u'Tmin', u'Tavg', u'Depart', u'DewPoint',
                         u'WetBulb', u'Heat', u'Cool', u'Snowfall',
                         u'PrecipTotal', u'StnPressure', u'SeaLevel',
                         u'ResultSpeed', u'ResultDir', u'AvgSpeed', u'Water1')

WEATHER_PHENOMENA = ('BCFG', 'BLDU', 'BLSN', 'BR', 'DU', 'DZ', 'FG', 'FG+',
                     'FU', 'FZDZ', 'FZFG', 'FZRA', 'GR', 'GS', 'HZ', 'MIFG',
                     'PL', 'PRFG', 'RA', 'SG', 'SN', 'SQ', 'TS', 'TSRA',
                     'TSSN', 'UP', 'VCFG', 'VCTS')

In [4]:
def haversine_distance(lat1, lon1, lat2, lon2):
    r_earth = 6371.
    dlat = np.abs(lat1-lat2)*np.pi/180.
    dlon = np.abs(lon1-lon2)*np.pi/180.
    lat1 *= np.pi/180.
    lat2 *= np.pi/180.
    dist = 2. * r_earth * np.arcsin(
                            np.sqrt(
                                np.sin(dlat/2.)**2 +
                                    np.cos(lat1) * np.cos(lat2) *
                                    np.sin(dlon/2.)**2))
    return dist

In [6]:
def lat_lon_box(lat, lon, dist):
    r_earth = 6371.
    d_2r = dist/(2.*r_earth)
    dlat = 2. * (d_2r)
    dlon = 2. * np.arcsin((np.sin(d_2r))/(np.cos(lat)))
    dlat *= 180./np.pi
    dlon *= 180./np.pi
    return abs(dlat), abs(dlon) 

In [13]:
def feature_extraction():
    spray_df = pd.read_csv('spray.csv')

    spray_lat_lon_list = []
    for idx, row in spray_df.iterrows():
        spray_lat_lon_list.append((row['Latitude'], row['Longitude']))

    weather_features = []
    cumu_labels = ('Tmax', 'Tmin', 'PrecipTotal')
    cumu_features = {}
    cumu_total = 0
    current_year = -1
    with open('weather.csv', 'rt') as wfile:
        wcsv = csv.reader(wfile, delimiter=',')
        weather_labels = next(wcsv)
        for row in wcsv:
            rowdict = dict(zip(weather_labels, row))
            rowdict['Date'] = parse(rowdict['Date'])
            current_date = rowdict['Date']
            if current_date.year != current_year:
                current_year = current_date.year
                cumu_features = {k: 0 for k in cumu_labels}
                cumu_total = 0
            for k in WEATHER_VARS_WITH_M_T:
                if k in rowdict:
                    rowdict[k] = rowdict[k].replace('M', 'nan')
                    rowdict[k] = rowdict[k].replace('T', '0.0')
            for k in rowdict:
                if rowdict[k] == '-':
                    rowdict[k] = 'nan'
                if type(rowdict[k]) == str:
                    rowdict[k] = rowdict[k].strip()
            for ph in WEATHER_PHENOMENA:
                rowdict['wp%s' % ph] = '0'
            for ph in rowdict['CodeSum'].split():
                if ph in WEATHER_PHENOMENA:
                    rowdict['wp%s' % ph] = '1'
            for lab in cumu_labels:
                _tmp = float(rowdict[lab])
                if not np.isnan(_tmp):
                    cumu_features[lab] += _tmp
            cumu_total += 1
            for lab in ('Tmax', 'Tmin', 'PrecipTotal'):
                rowdict['%s_cumu' % lab] = cumu_features[lab] / cumu_total
            weather_features.append(rowdict)
#            print('\n'.join(['%s: %s' % (k, rowdict[k]) for k in rowdict]))
#            exit(0)
    for ph in WEATHER_PHENOMENA:
        weather_labels.append('wp%s' % ph)
    for lab in cumu_labels:
        weather_labels.append('%s_cumu' % lab)


    for prefix in 'train', 'test':
        with open('%s.csv' % prefix, 'rt') as csvfile:
            outfile = open('%s_full.csv' % prefix, 'w', newline='')
            csv_reader = csv.reader(csvfile)
            labels = next(csv_reader)

            out_labels = labels +\
                         ['n_spray_%d' % x for x in range(1,11)]
            for lab in weather_labels:
                if lab == 'Date':
                    continue
                out_labels.append(lab)
                csv_writer = csv.writer(outfile)
            csv_writer.writerow(out_labels)

            for idx, row in enumerate(csv_reader):
                if idx % 1000 == 0:
                    print('processed %d' % idx)
#                if idx > 100:
#                    exit(0)
                row_dict = dict(zip(labels, row))

                current_date = parse(row_dict['Date'])
                cur_lat = float(row_dict['Latitude'])
                cur_lon = float(row_dict['Longitude'])

                for idx in range(1, 11):
                    row_dict['n_spray_%d' % idx] = 0
                dlat, dlon = lat_lon_box(cur_lat, cur_lon, 1.5)
                for slat, slon in spray_lat_lon_list:
#                    print(dlat, dlon, abs(slat-cur_lat), abs(slon-cur_lon))
                    if abs(slat-cur_lat) > dlat or abs(slon-cur_lon) > dlon:
                        continue
                    sdist = haversine_distance(cur_lat, cur_lon, slat, slon)
                    for idx in range(1,11):
                        if sdist < idx/10.0:
                            row_dict['n_spray_%d' % idx] += 1

                for lab in ['Tmax_cumu', 'Tmin_cumu', 'PrecipTotal_cumu']:
                    row_dict[lab] = 0
                most_recent = 1000000
                most_recent_w = weather_features[0]
                for wfeat in weather_features:
                    wdate = wfeat['Date']
                    if current_date.year != wdate.year:
                        continue
                    wdur = abs((current_date - wdate).days)
                    if wdur < most_recent:
                        most_recent = wdur
                        most_recent_w = wfeat
                for lab in weather_labels:
                    if lab == 'Date':
                        continue
                    row_dict[lab] = most_recent_w[lab]
                row_val = [row_dict[col] for col in out_labels]
                csv_writer.writerow(row_val)
#                outfile.flush()
#                print('\n'.join(['%s: %s' % (k, row_dict[k]) for k in row_dict]))
#                exit(0)
    return

In [14]:
if __name__ == '__main__':
    feature_extraction()

processed 0
processed 1000
processed 2000
processed 3000
processed 4000
processed 5000
processed 6000
processed 7000
processed 8000
processed 9000
processed 10000
processed 0
processed 1000
processed 2000
processed 3000
processed 4000
processed 5000
processed 6000
processed 7000
processed 8000
processed 9000
processed 10000
processed 11000
processed 12000
processed 13000
processed 14000
processed 15000
processed 16000
processed 17000
processed 18000
processed 19000
processed 20000
processed 21000
processed 22000
processed 23000
processed 24000
processed 25000
processed 26000
processed 27000
processed 28000
processed 29000
processed 30000
processed 31000
processed 32000
processed 33000
processed 34000
processed 35000
processed 36000
processed 37000
processed 38000
processed 39000
processed 40000
processed 41000
processed 42000
processed 43000
processed 44000
processed 45000
processed 46000
processed 47000
processed 48000
processed 49000
processed 50000
processed 51000
processed 52000
pr

In [7]:
SPECIES = ['CULEX ERRATICUS', 'CULEX PIPIENS', 'CULEX PIPIENS/RESTUANS',
           'CULEX RESTUANS', 'CULEX SALINARIUS', 'CULEX TARSALIS',
           'CULEX TERRITANS']

In [8]:
def clean_data(df):
    df['Species'] = df['Species'].map({k: n for (n, k) in enumerate(SPECIES)})
    n_null = (df['Species'].isnull()).sum()
    nan_species = np.random.random_integers(0, len(SPECIES)-1, size=(n_null,))
    if n_null > 0:
        df.loc[df['Species'].isnull(), 'Species'] = nan_species

    df['is_sat_trap'] = df['Trap'].apply(lambda x: len(x[4:]) > 0).astype(int)
    df['Trap'] = df['Trap'].apply(lambda x: x[1:4]).astype(int)

    df = df.drop(labels=['Date', 'Address', 'Street',
                         'AddressNumberAndStreet', 'CodeSum', 'SnowFall',
                         'Water1', 'Station', 'Depth', 'wpBCFG', 'wpBLDU',
                         'wpBLSN', 'wpDU', 'wpFG+', 'wpFU', 'wpFZDZ',
                         'wpFZFG', 'wpFZRA', 'wpGR', 'wpGS', 'wpMIFG', 'wpPL',
                         'wpPRFG', 'wpSG', 'wpSN', 'wpSQ', 'wpTSSN', 'wpUP',
                         'wpVCFG', 'AddressAccuracy', 'WetBulb', 
                         'StnPressure', 'Latitude', 'Longitude'], axis=1)
    return df

In [9]:
def load_data(do_plots=False):
    train_df = pd.read_csv('train_full.csv')
    test_df = pd.read_csv('test_full.csv')
    submit_df = pd.read_csv('sampleSubmission.csv')

    train_df = clean_data(train_df)
    test_df = clean_data(test_df)

    print(submit_df.dtypes)

    for col in test_df.columns:
        if (test_df[col].isnull()).sum() > 0:
            print(col, test_df[col].dtype)
#    print(sorted(train_df['is_sat_trap'].unique()))
#    print(sorted(test_df['Species'].unique()))
    if do_plots:
        from plot_data import plot_data
        plot_data(train_df, prefix='train_html')
        plot_data(test_df, prefix='test_html')

    features = train_df.drop(labels=['NumMosquitos', 'WnvPresent'],
                           axis=1).columns

    xtrain = train_df.drop(labels=['NumMosquitos', 'WnvPresent'],
                           axis=1).values
    ytrain = train_df[['NumMosquitos', 'WnvPresent']].values
    xtest = test_df.drop(labels=['Id'], axis=1).values
    ytest = submit_df
    return xtrain, ytrain, xtest, ytest, features

In [10]:
if __name__ == '__main__':
    xtrain, ytrain, xtest, ytest, features = load_data(do_plots=False)
    print([df.shape for df in (xtrain, ytrain, xtest, ytest)])

C:\Anaconda3 2.1.0\lib\site-packages\pandas\io\parsers.py:1159: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
C:\Anaconda3 2.1.0\lib\site-packages\pandas\io\parsers.py:1159: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Id            int64
WnvPresent    int64
dtype: object
[(10506, 39), (10506, 2), (116293, 39), (116293, 2)]


In [11]:
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier,\
                             GradientBoostingRegressor, \
                             GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
#from sklearn.metrics.roc_curve
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.grid_search import GridSearchCV

In [22]:
def transform_to_log(y):
    #return np.log1p(y)
    return np.sqrt(3/8+y)

In [23]:
def transform_from_log(ly):
    #return np.round(np.expm1(ly)).astype(int)
    return np.round(np.square(ly)-3/8).astype(int)

In [24]:
def scorer(estimator, X, y):
    ypred = estimator.predict_proba(X)
    return 1.0/roc_auc_score(y, ypred[:, 1])

In [25]:
def train_nmosq_model(model, xtrain, ytrain, do_grid_search=False):
    xTrain, xTest, yTrain, yTest = train_test_split(xtrain,
                                                    ytrain[:,0],
                                                    test_size=0.5)
    n_est = [10, 20]
    m_dep = [2, 3, 4, 5, 6, 7, 10]

    if do_grid_search:
        model = GridSearchCV(estimator=model,
                                    param_grid=dict(n_estimators=n_est,
                                                    max_depth=m_dep),
                                    scoring=scorer,
                                    n_jobs=-1, verbose=1)
    model.fit(xTrain, yTrain)
    print(model.score(xTest, yTest))
    if hasattr(model, 'best_params_'):
        print(model.best_params_)

In [18]:
def train_has_wnv_model(model, xtrain, ytrain, do_grid_search=False,
                        feature_list=None):
    xTrain, xTest, yTrain, yTest = train_test_split(xtrain,
                                                    ytrain[:,1],
                                                    test_size=0.5)
    n_est = [10, 20]
    m_dep = [2, 3, 4, 5, 6, 7, 10]

    if do_grid_search:
        model = GridSearchCV(estimator=model,
                                    param_grid=dict(n_estimators=n_est,
                                                    max_depth=m_dep),
                                    scoring=scorer,
                                    n_jobs=-1, verbose=1)
    model.fit(xTrain, yTrain)
    ypred = model.predict_proba(xTest)
    print(roc_auc_score(yTest, ypred[:, 1]))
    if hasattr(model, 'best_params_'):
        print(model.best_params_)
    if hasattr(model, 'feature_importances_') and feature_list is not None:
        print('\n'.join(['%s: %s' % (k, v) for (k,v) in sorted(zip(feature_list, 
               model.feature_importances_), key=lambda x: x[1])]))
    return

In [33]:
def prepare_submission(model, xtrain, ytrain, xtest, ytest, feature_list=None):
    model.fit(xtrain, ytrain)
    if hasattr(model, 'feature_importances_') and feature_list is not None:
        print('\n'.join(['%s: %s' % (k, v) for (k,v) in sorted(zip(feature_list, 
               model.feature_importances_), key=lambda x: x[1])]))
    ypred = model.predict_proba(xtest)
    ytest.loc[:, 'WnvPresent'] = ypred[:, 1]
    ytest['Id'] = ytest['Id'].astype(int)
    ytest.to_csv('k_jun3_1.csv', index=False)

In [39]:
np.random.seed(16809)
def my_model():
    xtrain, ytrain, xtest, ytest, features = load_data()

#    ytrain = transform_to_log(ytrain)
#
#    mosq_model = GradientBoostingRegressor(loss='ls', verbose=1, max_depth=7,
#                                        n_estimators=20)
#    train_nmosq_model(mosq_model, xtrain, ytrain, do_grid_search=False)

    model = GradientBoostingClassifier(verbose=1, max_depth=3, learning_rate=0.0975,
                                       n_estimators=200)
    print(model)

    train_has_wnv_model(model, xtrain, ytrain, do_grid_search=False, 
                        feature_list=features)

    prepare_submission(model, xtrain, ytrain[:, 1], xtest, ytest, 
                       feature_list=features)
    

    return

In [40]:
if __name__ == '__main__':
    my_model()

Id            int64
WnvPresent    int64
dtype: object
GradientBoostingClassifier(init=None, learning_rate=0.0975, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2, n_estimators=200,
              random_state=None, subsample=1.0, verbose=1,
              warm_start=False)
      Iter       Train Loss   Remaining Time 
         1           0.4071            6.29s
         2           0.3969            4.42s
         3           0.3888            3.95s
         4           0.3810            4.16s
         5           0.3734            3.81s
         6           0.3683            3.67s
         7           0.3631            3.83s
         8           0.3590            3.46s
         9           0.3558            3.39s
        10           0.3523            3.63s
        20           0.3306            3.16s
        30           0.3170            2.86s
        40           0.3090            2.61s
       